In [ ]:
import ixmp
import pyam
import nomenclature

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")
eu_climate_rp = nomenclature.RegionProcessor.from_directory("../mappings")

In [ ]:
source_db = ixmp.Platform("ecemf-internal")

In [ ]:
props = source_db.scenario_list()

In [ ]:
props.model.unique()

In [ ]:
list()

In [ ]:
props[props.model == "REMIND 2.1"]

In [ ]:
scenario_mapping = {
#    "REMIND 2.1": [
# done        "DIAG-C400-lin",
# done        "DIAG-NPI",
# done        "DIAG-C400-lin-LimBio",
# done        "DIAG-C400-lin-HighEff",
# done        "DIAG-C400-lin-LimNuclear",
# done        "DIAG-NZero"
#    ],
#    "IMAGE 3.2": [
# done        "DIAG-NPI",
# done        "DIAG-C400-lin",
#    ],
#    "OSeMBE v1.0.0": [
# done       "DIAG-NPI",
# done       "DIAG-C400-lin",
#    ],
#    "PROMETHEUS 1.2": [
# done         "DIAG-NPI",
# done         "DIAG-C400-lin",
#    ],
#    "PRIMES 2022": [
# done         "DIAG-NPI",
# done         "DIAG-C400-lin",
#    ],
#}

In [ ]:
scenario_selection = list()
for model, scenarios in scenario_mapping.items():
    for s in scenarios:
        scenario_selection.append((model, s))

In [ ]:
scenario_selection

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=source_db, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
df.rename(unit={
    "Mt CO2e/yr": "Mt CO2-equiv/yr",
    "Gpkm/yr": "billion pkm/yr",
    "Gtkm/yr": "billion tkm/yr",
    "bn m2/yr": "billion m2/yr",
}, inplace=True)

In [ ]:
remind_mapping = {}
for key, value in eu_climate_rp.mappings["REMIND 2.1"].rename_mapping.items():
    remind_mapping[f"REMIND 2.1|{key}"] = value

df.rename(region=remind_mapping, inplace=True)

In [ ]:
image_mapping = {}
for key, value in eu_climate_rp.mappings["IMAGE 3.2"].rename_mapping.items():
    image_mapping[f"IMAGE 3.2|{key}"] = value
df.rename(region=image_mapping, inplace=True)

In [ ]:
osembe_mapping = {}
for key, value in eu_climate_rp.mappings["ALADIN 1.0"].rename_mapping.items():
    osembe_mapping[f"OSeMBE v1.0|{key}"] = value
df.rename(region=osembe_mapping, inplace=True)

In [ ]:
df.filter(region=['Baltics', 'EU27 & EFTA', 'Scandinavia', '* (IP)'], keep=False, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(_df.data)
    scen.commit(f"Migrated from H2020 ECEMF (project-internal) database")
    scen.set_as_default()

    scen.set_meta({"Source": "H2020 ECEMF"})